In [0]:
# 01_bronze_stream_encounters_autoloader.ipynb
# Ingest CSV encounter files into a Bronze Delta table with Change Data Feed enabled.

from pyspark.sql.types import *

# Paths and table names
BRONZE_DB = "kardia_bronze"
BRONZE_ENCOUNTERS_TABLE = f"{BRONZE_DB}.bronze_encounters"
RAW_PATH = "dbfs:/kardia/raw/encounters/"
BRONZE_PATH = "dbfs:/kardia/bronze/bronze_encounters"
SCHEMA_PATH = "dbfs:/kardia/_schemas/encounters"
CHECKPOINT_PATH = "dbfs:/kardia/_checkpoints/bronze_encounters"
BAD_PATH = "dbfs:/kardia/_quarantine/raw/bad_encounters"

# Minimize shuffle overhead for small test datasets
spark.conf.set("spark.sql.shuffle.partitions", "1")

In [0]:
# Explicit schema defines data contract and improves performance
encounters_schema = StructType(
    [
        StructField("ID", StringType(), False),
        StructField("DATE", DateType(), True),
        StructField("PATIENT", StringType(), False),
        StructField("CODE", StringType(), True),
        StructField("DESCRIPTION", StringType(), True),
        StructField("REASONCODE", StringType(), True),
        StructField("REASONDESCRIPTION", StringType(), True)
    ]
)

In [0]:
# 1. Make sure the Bronze DB and table exists
spark.sql(f"CREATE DATABASE IF NOT EXISTS {BRONZE_DB}")

spark.sql(
    f"""
    CREATE TABLE IF NOT EXISTS {BRONZE_ENCOUNTERS_TABLE}
    USING DELTA
    COMMENT 'Bronze table for batch Auto Loader ingest of encounter records.'
    LOCATION '{BRONZE_PATH}'
    TBLPROPERTIES ('delta.enableChangeDataFeed' = 'true')
    """
)

In [0]:
# 2. Start Auto Loader stream
stream = (
    spark.readStream
        .format("cloudFiles")
        .option("cloudFiles.format", "csv")
        .option("cloudFiles.includeExistingFiles", "true")
        .option("cloudFiles.schemaLocation", SCHEMA_PATH)
        .option("header", "true")
        .option("badRecordsPath", BAD_PATH)
        .option("rescuedDataColumn", "_rest")
        .schema(encounters_schema)
        .load(RAW_PATH)

        .writeStream
        .format("delta")
        .option("mergeSchema", "true")
        .option("checkpointLocation", CHECKPOINT_PATH)
        .outputMode("append")
        .trigger(availableNow=True)
        .start(BRONZE_PATH)
)

stream.awaitTermination()
print(f"Bronze ingest complete: from {RAW_PATH} to {BRONZE_PATH}")

In [0]:
# 3. Stream finished - confirm results
print(f"Loaded data from {RAW_PATH} to {BRONZE_PATH}")

df = spark.read.format("delta").load(BRONZE_PATH)
print(f"Row count: {df.count()}")
display(df.limit(5))

# Show Delta Lake history to verify CDF and ingest details
history_df = spark.sql(
    f"""
    DESCRIBE HISTORY delta.`{BRONZE_PATH}`
    """
).select("version", "timestamp", "operation", "operationParameters")

display(history_df.limit(5))